<H1>Лабораторная 5</H1>

Задание: Разработать алгоритм для подготовки данных, необходимых для определения коллизии двух произвольных объектов, сохранить их в файл и загрузить эти данные в оперативную память.

In [9]:
import pathlib
import math

In [10]:
class GameObject:
    """
    Игровой объект с заданной формой по координатам точек и типом.\n
    Параметры объекта:\n
    points: List[tuple[int, int]] - список пар координат точек, которыми задается форма объекта.\n
    type: str - название типа объекта {ship, torpedo, asteroid}.\n
    """
    def __init__(self, points: list[tuple[int, int]], object_type: str):
        self.points = points
        self.type = object_type

In [11]:
def segments_intersect(points: tuple[tuple[int, int], tuple[int, int], tuple[int, int], tuple[int, int]]) -> bool:
    """
    Проверка пересечения двух отрезков.\n
    Принимаемые значения:\n
    points: tuple[tuple[int, int], tuple[int, int], tuple[int, int], tuple[int, int]] - массив из 4 пар координат точек, которые соответствуют концам двух отрезков, пересечение которых проверяется.\n
    Отрезки представляются попарно: первая пара точек является соответственно началом и концом первого отрезка, вторая пара точек является соответственно началом и концом второго отрезка.\n
    Возвращаемые значения:\n
    True - в случае, если два отрезка пересекаются.\n
    False - в случае, если два отрезка не пересекаются.
    """
    p1 = points[0]
    p2 = points[1]
    p3 = points[2]
    p4 = points[3]

    def ccw(points: tuple[tuple[int, int], tuple[int, int], tuple[int, int]]):
        """
        Вспомогательная функция для проверки пересечения двух объектов.\n
        Вычисляет векторное произведение векторов.\n
        Принимаемые значения:\n
        points: tuple[tuple[int, int], tuple[int, int], tuple[int, int]] - массив из 3 пар координат точек, из которых строятся векторы.
        """
        P1 = points[0]
        P2 = points[1]
        P3 = points[2]
        return (P3[1]-P1[1])*(P2[0]-P1[0]) - (P2[1]-P1[1])*(P3[0]-P1[0])

    def on_segment(points: tuple[tuple[int, int], tuple[int, int], tuple[int, int]]) -> bool:
        """
        Вспомогательная функция для проверки пересечения двух объектов.\n
        Проверяет, являются ли точки коллинеарными(находящимися на одной линии).\n
        Принимаемые значения:\n
        points: tuple[tuple[int, int], tuple[int, int], tuple[int, int]] - массив из 3 пар координат точек, из которых строятся векторы.
        """
        P1 = points[0]
        P2 = points[1]
        P3 = points[2]
        return min(P1[0], P2[0]) <= P3[0] <= max(P1[0], P2[0]) and min(P1[1], P2[1]) <= P3[1] <= max(P1[1], P2[1])

    ccw1 = ccw((p3, p4, p1))
    ccw2 = ccw((p3, p4, p2))
    ccw3 = ccw((p1, p2, p3))
    ccw4 = ccw((p1, p2, p4))

    if (ccw1 * ccw2 < 0) and (ccw3 * ccw4 < 0):
        return True

    if ccw1 == ccw2 == ccw3 == ccw4 == 0:
        return on_segment(p1, p2, p3) or on_segment(p1, p2, p4) or on_segment(p3, p4, p1) or on_segment(p3, p4, p2)

    return False


In [12]:
def check_collision(moving_object: GameObject, static_object: GameObject, velocity: tuple[int, int]):
    """
    Функция для проверки коллизии двух объектов произвольной геометрической формы с заданным вектором скорости движущегося объекта.\n
    Использует функцию segments_intersect для проверки пересечения отрезка пути каждой точки движущегося объекта с каждым отрезком формы статичного объекта.\n
    Принимаемые значения:\n
    moving_object: GameObject - передвигаемый объекта типа GameObject.\n
    static_object: GameObject - статичный объект типа GameObject, с которым проверяется коллизия движущегося объекта.\n
    velocity: tuple[int, int] - вектор скорости передвигаемого объекта.\n
    Возвращаемые значения:
    (True, tuple[int, int]) - в случае, если коллизия обнаружена (пересечение какого-либо отрезка пути точки движущегося объекта с каким-либо отрезком формы статичного объекта), возвращает True и точку, которая вызвала коллизию.\n
    (False, None) - в случае, если коллизия не была обнаружена.
    """
    st_obj_points = static_object.points

    for moving_object_point in moving_object.points:
        moving_object_current_point = moving_object_point
        moving_object_future_point = (moving_object_current_point[0] + velocity[0], moving_object_current_point[1] + velocity[1])
        for j in range(len(st_obj_points)):
            static_object_first_point = st_obj_points[j]
            static_object_second_point = st_obj_points[(j + 1) % len(static_object.points)]
            if segments_intersect((moving_object_current_point, moving_object_future_point, static_object_first_point, static_object_second_point)):
                return (True, moving_object_future_point)
    
    return (False, None)

In [13]:
def check_collision_in_all_directions(moving_object: GameObject, static_object: GameObject, velocity: int, angle_step: int):
    """
    Функция для проверки коллизии двух объектов произвольной геометрической формы с заданным модулем скорости.\n
    Использует функцию check_collision для проверки коллизии в выбранном направлении вектора скорости.\n
    Вектор скорости выбирается в соответствии с переданным модулем скорости и шагом угла поворота вектора.\n
    Принимаемые значения:\n
    moving_object: GameObject - передвигаемый объекта типа GameObject.\n
    static_object: GameObject - статичный объект типа GameObject, с которым проверяется коллизия движущегося объекта.\n
    velocity: int - модуль вектора скорости движимого объекта.\n
    angle_step: int - шаг поворота вектора скорости движимого объекта.\n
    Возвращаемые значения:\n
    tuple[int, int, int, int] - данные о коллизии, состоящие из координат точки, которая вызвала коллизию, и из двух составляющих вектора скорости по осям x и y.
    """

    collisions = []

    for current_angle in range(0, 360, angle_step):
        rad = math.radians(current_angle)
        velocity_x = velocity * math.cos(rad)
        velocity_y = velocity * math.sin(rad)
        is_collision_detected, collision_point = check_collision(moving_object, static_object, (velocity_x, velocity_y))
        if is_collision_detected:
            collisions.append((collision_point[0], collision_point[1], velocity_x, velocity_y))
    return collisions

In [14]:
def write_collisions_to_file(collisions: tuple[int, int, int, int], moving_object_type: str, static_object_type: str):
    """
    Функция для записи информации о коллизиях в соответствующий файл типа txt.\n
    Принимаемые значения:\n
    collisions: tuple[int, int, int, int] - информация о коллизиях.\n
    moving_object_type: str - тип движимого объекта.\n
    static_object_type: str - тип статичного объекта.
    """
    directory = pathlib.Path('game_collisions')
    directory.mkdir(exist_ok=True)
    
    filename = f"{moving_object_type}_{static_object_type}_collision.txt"
    filepath = directory / filename
    
    with open(filepath, 'w', encoding='utf-8') as f:
        for dx, dy, vx, vy in collisions:
            f.write(f"{dx},{dy},{vx},{vy}\n")

In [ ]:
def build_collision_tree(moving_object_type: str, static_object_type: str) -> dict:
    """
    Функция для чтения данных о коллизии из txt файла и создания древовидной структуры.\n
    Название файла определяется типами движущегося и статичного объектов в формате:\n
    {moving_object_type}_{static_object_type}_collision.txt\n
    Принимаемые значения:\n
    moving_object_type: str - тип движущегося объекта.\n
    static_object_type: str - тип статичного объекта.\n
    Возвращаемые значения:\n
    None - если файл не был найден.\n
    dict{
        "moving_object": str, 
        "static_object": str, 
        "collisions": [
            dict
            {
                "dx":float,
                "dy":float,
                "vx":float,
                "vy":float 
            }
            ...
        ]
    } - если файл найден, то возвращает дерево, представленное в виде словаря с данными - тип движимого объекта, тип статичного объекта и информация о коллизиях этих двух объектов.
    """
    directory = pathlib.Path('game_collisions')

    filename = f"{moving_object_type}_{static_object_type}_collision.txt"
    filepath = directory / filename

    if not filepath.exists:
        return None

    collisions = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for string in f:
            dx, dy, vx, vy = map(float, string.strip().split(','))
            collisions.append(
                {
                    "dx":dx,
                    "dy":dy, 
                    "vx":vx, 
                    "vy":vy
                }
            )

    collision_tree = {
        "moving_object": moving_object_type,
        "static_object": static_object_type,
        "collisions": collisions
    }

    return collision_tree

In [16]:
ship = GameObject([(-2,-2), (-1,-2), (-1,-1), (-2,-1)], "ship")
asteroid = GameObject([(0,0), (1,1), (1,2), (0,3), (-1,2), (-1,1)], "asteroid")

ship_velocity = 10

angle_step = 10

collisions = check_collision_in_all_directions(ship, asteroid, ship_velocity, angle_step)

write_collisions_to_file(collisions, ship.type, asteroid.type)

build_collision_tree(ship.type, asteroid.type)

{'moving_object': 'ship',
 'static_object': 'asteroid',
 'collisions': [{'dx': 6.660254037844387,
   'dy': 3.999999999999999,
   'vx': 8.660254037844387,
   'vy': 4.999999999999999},
  {'dx': 5.66044443118978,
   'dy': 5.4278760968653925,
   'vx': 7.66044443118978,
   'vy': 6.4278760968653925},
  {'dx': 4.427876096865393,
   'dy': 5.66044443118978,
   'vx': 6.427876096865393,
   'vy': 7.66044443118978},
  {'dx': 3.000000000000001,
   'dy': 6.6602540378443855,
   'vx': 5.000000000000001,
   'vy': 8.660254037844386},
  {'dx': 1.4202014332566884,
   'dy': 7.396926207859083,
   'vx': 3.4202014332566884,
   'vy': 9.396926207859083},
  {'dx': 0.7364817766693041,
   'dy': 7.8480775301220795,
   'vx': 1.7364817766693041,
   'vy': 9.84807753012208},
  {'dx': -0.9999999999999993,
   'dy': 8.0,
   'vx': 6.123233995736766e-16,
   'vy': 10.0}]}